In [1]:
include("src\\building_tree.jl")
include("src\\utilities.jl")
include("src\\merge.jl")
using Debugger
using DataFrames, Random
using DataStructures
using StatsBase
using Statistics
using LinearAlgebra
using LazySets
using Polyhedra

In [2]:
dataSetName = "iris"
include("./data/" * dataSetName * ".txt")
# Ramener chaque caractéristique sur [0, 1]
reducedX = Matrix{Float64}(X)
for j in 1:size(X, 2)
    reducedX[:, j] .-= minimum(X[:, j])
    reducedX[:, j] ./= maximum(X[:, j])
end

train, test = train_test_indexes(length(Y))
X_train = reducedX[train,:]
Y_train = Y[train]
X_test = reducedX[test,:]
Y_test = Y[test]
classes = unique(Y)

println(" (train size ", size(X_train, 1), ", test size ", size(X_test, 1), ", ", size(X_train, 2), ", features count: ", size(X_train, 2), ")")

# Temps limite de la
time_limit = 10
D = 3
x = X_train
y = Y_train

 (train size 120, test size 30, 4, features count: 4)


120-element Vector{Int64}:
 3
 1
 3
 1
 1
 1
 1
 1
 2
 1
 ⋮
 3
 1
 2
 3
 1
 1
 1
 2
 1

In [3]:
minimum(X_train), maximum(X_train)

(0.0, 1.0)

In [4]:
num_clusters = 4
centroids = [rand(4) for i in 1:num_clusters]
clusters_assignment = Vector{Int64}([])
closest_distance = Inf
for i in 1:size(X_train, 1)
    x_test = X_train[i,:]
    cluster = 0
    for (j, centroid) in enumerate(centroids)
        distance = euclidean(x_test, centroid)
        if distance < closest_distance
            cluster =j
            closest_distance = distance
        end
    end
    push!(clusters_assignment, cluster)
end

In [5]:
using Clustering

In [6]:
number_clusters = 5
R = Clustering.kmeans(transpose(X_train), number_clusters)
a = assignments(R)
c = counts(R)
M = R.centers

4×5 Matrix{Float64}:
 0.443237  0.115385   0.303241   0.711111  0.207602
 0.299819  0.419872   0.802083   0.454167  0.607456
 0.575166  0.0756193  0.0889831  0.800565  0.0811775
 0.539855  0.0352564  0.0729167  0.834722  0.0767544

In [7]:
clusters = Vector{Cluster}([])
for cluster_interest in 1:number_clusters
    count_classes_in_clusters = StatsBase.countmap(y[findall(x->x==cluster_interest, a)])
    y_cluster = y[findall(x->x==2, a)]
    df = DataFrame()
    df."classes" = collect(keys(count_classes_in_clusters))
    df."count_classes" = collect(values(count_classes_in_clusters))
    df = sort(df, [:count_classes], rev=true)
    number_classes = size(df, 1)
    if number_classes > 1
        println(cluster_interest)
    end
    points_cluster = findall(x->x==cluster_interest, a)
    cluster_group = Cluster(points_cluster[1], X_train, Y_train)
    for point_cluster in points_cluster[2:end]
        new_cluster = Cluster(point_cluster, X_train, Y_train)
        merge!(cluster_group, new_cluster)
    end
    push!(clusters, cluster_group)
end
clusterId = a
df_clusters = DataFrame()
df_clusters."cluster_id" = collect(keys(StatsBase.countmap(clusterId)))
df_clusters."number_elements" = collect(values(StatsBase.countmap(clusterId)))
higher_than_threshold(value::Int64) = value >= 10
clusters_to_treat = filter(:"number_elements"=> higher_than_threshold, df_clusters)."cluster_id"

1


4

5-element Vector{Int64}:
 5
 4
 2
 3
 1

In [23]:
df_clusters_bis = DataFrame(cluster_id=Int64[], number_elements=Int64[])

Row,cluster_id,number_elements
,Int64,Int64


In [24]:
push!(df_clusters_bis, [2,3])

Row,cluster_id,number_elements
,Int64,Int64
1,2,3


In [25]:
df_clusters_bis

Row,cluster_id,number_elements
,Int64,Int64
1,2,3


In [18]:
vcat(df_clusters, df_clusters_bis)

Row,cluster_id,number_elements
,Int64,Int64
1,5,19
2,4,30
3,2,13
4,3,12
5,1,46
6,6,2
7,7,4
8,8,6
9,9,8


In [64]:
for cluster_interest in clusters_to_treat 
    count_classes_in_clusters = StatsBase.countmap(y[findall(x->x==cluster_interest, clusterId)])
    y_cluster = y[findall(x->x==cluster_interest, clusterId)]
    df = DataFrame()
    df."classes" = collect(keys(count_classes_in_clusters))
    df."count_classes" = collect(values(count_classes_in_clusters))
    df = sort(df, [:count_classes], rev=true)
    main_class = 0
    if size(df)[1]>1
        main_class = df[1,1]
        print("several classes in cluster")
    else
        println("only one class in cluster")
        println(", we are not touching cluster ", cluster_interest)
        main_class = df[1,1]
    end   
    points_in_cluster = findall(x->x==cluster_interest, clusterId)
    points_reckon_with = Vector{Int64}([])
    points_sideline = Vector{Int64}([])
    for (i, point) in enumerate(points_in_cluster)
        if y[point] != main_class
            push!(points_sideline,i )
        else
            push!(points_reckon_with, i)
        end
    end
    matrix_interest = clusters[cluster_interest].x
    size(matrix_interest, 1)
    vectors_cluster = [matrix_interest[i,:] for i in 1:size(matrix_interest,1)]
    vectors_cluster[points_reckon_with]
    hull = LazySets.convex_hull(vectors_cluster[points_reckon_with])
    for i in 1:length(points_sideline)
        println(i, " ",element(Singleton(vectors_cluster[points_sideline[i]])) ∈ VPolytope(hull))
        if element(Singleton(vectors_cluster[points_sideline[i]])) ∈ VPolytope(hull)
            push!(points_reckon_with,  points_sideline[i])
        end
    end
    sort!(points_reckon_with)
    points_to_be_taken_out_cluster = points_sideline[points_sideline .∉ Ref(points_reckon_with)]

    new_clusters = Vector{Cluster}([])
    point_cluster  = points_reckon_with[1]
    trimmed_cluster = Cluster(point_cluster, matrix_interest, y_cluster)
    for point_cluster in points_reckon_with[2:end]
        new_cluster = Cluster(point_cluster, matrix_interest, y_cluster)
        merge!(trimmed_cluster, new_cluster)
    end

    for point_cluster in points_to_be_taken_out_cluster
        new_cluster = Cluster(point_cluster, matrix_interest, y_cluster)
        push!(clusters, new_cluster)
    end
    clusters[cluster_interest] = trimmed_cluster
    print(length(clusters))
end

only one class in cluster
, we are not touching cluster 5


glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
5

several classes in clusterglp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
glp_simplex: unable to recover undefined or non-optimal solution
1 false
2 false
3 false
4 false
5 false
6 false
7 false
8 false
9 false
10 false
11 false
12 false
17only one class in cluster
, 

In [51]:

for element in findall(x->x==cluster_interest, a)
    push!(clusters, Cluster(element,X_train, Y_train, ))
end
main_cluster = clusters_current_group[1]
for element in clusters_current_group[2:end]
    push!()

In [1]:
num_clusters = 10
centroids = [rand(4) for i in 1:num_clusters]
for i in 1:1
    clusters_assignment = Vector{Int64}([])
    for i in 1:size(X_train, 1)
        closest_distance = 100
        x_test = X_train[i,:]
        cluster = 0
        for (j, centroid) in enumerate(centroids)
            distance = euclidean(x_test, centroid)
            if distance < closest_distance
                cluster =j
                closest_distance = distance
            end
        end
        push!(clusters_assignment, cluster)
    end
    new_centroids = Vector{Vector{Float64}}([])
    for i in 1:num_clusters
        df = DataFrame()
        df."id" = collect(1:size(X_train,1))
        df."cluster_assignment" = clusters_assignment
        cluster_value(value::Int64) = value == i
        new_centroid = vec(mean(X_train[filter(:"cluster_assignment"=>cluster_value, df)."id",:],dims=1))
        # print(new_centroid)
        push!(new_centroids, new_centroid)
    end
    centroids = new_centroids
end

UndefVarError: UndefVarError: X_train not defined